# Stage 4 — Feature Engineering Dataset (Booking + Grid Enrichment)

**Goal:** Create a single dataset at the *listing level* that contains:
- Booking cleaned listing data (from `booking_clean.parquet`)
- The same **1km `grid_id`** used in Stage 3 (exact formula)
- Grid-level enrichment features 
- Simple engineered features (log counts, distance caps, proximity flags, etc.)
- A `has_enrichment` flag and an **enriched-only** subset for ML

## 1) Imports + Parameters

In [0]:
from pyspark.sql import functions as F

BOOKING_PATH = "dbfs:/tmp/booking_clean/booking_clean.parquet"
GRID_ENRICH_PATH = "dbfs:/tmp/booking_grid_enrichment_final"
OUT_BASE = "dbfs:/tmp/booking_stage4"  

METERS_PER_DEG_LAT = 111320.0
GRID_M = 1000.0        # 1km
RADIUS_M = 800.0       


PROX_THRESHOLDS_M = [200.0, 500.0]


## 2) Load Booking Clean + Grid Enrichment

In [0]:
booking = spark.read.parquet(BOOKING_PATH)
enrich = spark.read.parquet(GRID_ENRICH_PATH).dropDuplicates(["grid_id"])

print("Booking rows:", booking.count())
print("Enriched grids:", enrich.count())

print("Booking columns:", len(booking.columns))
print("Enrichment columns:", len(enrich.columns))

display(booking.limit(5))
display(enrich.limit(5))

Booking rows: 3239391
Enriched grids: 2198
Booking columns: 23
Enrichment columns: 21


hotel_id url title city country location lat lon metro_railway_access description fine_print property_highlights property_information most_popular_facilities house_rules manager_language_spoken availability review_score manager_score number_of_reviews top_reviews property_surroundings images 8908679 https://www.booking.com/hotel/pl/komfortowe-noclegi.html Komfortowe Noclegi Szałsza Poland Ptasia 18, 42-677 Szałsza, Poland 50.3330617 18.7037022 true Providing a garden, Komfortowe Noclegi provides accommodations in Szałsza. This homestay offers free private parking, private check-in and check-out, and free Wifi. Górnik Zabrze is 5.6 miles away and Ruch Chorzów Stadium is 18 miles from the homestay.

Offering a balcony and garden views, the homestay includes 2 bedrooms, a living room, cable flat-screen TV, an equipped kitchen, and 1 bathroom with a shower. Towels and bed linen are featured in the homestay. The accommodation is non-smoking.

Stadion Śląski is 19 miles from the homestay, while Silesia City Center shopping mall is 21 miles away. Katowice Airport is 25 miles from the property. Please inform Komfortowe Noclegi of your expected arrival time in advance. You can use the Special Requests box when booking, or contact the property directly using the contact details in your confirmation. This property does not accommodate bachelor(ette) or similar parties. Quiet hours are between 21:00:00 and 06:00:00. When staying at the property with children, note that the property is legally obliged to apply standards for the protection of minors to determine the identity of the minors and their relationship with the adult they’re staying with. Free parking. Non-smoking rooms. Free Wifi null List(Free parking, Non-smoking rooms, Free Wifi) List(List(Check-in, From 3:00 PM to 10:00 PM You need to let the property know what time you'll be arriving in advance.), List(Check-out, From 12:00 AM to 11:30 AM), List(Cancellation/ prepayment, Cancellation and prepayment policies vary according to accommodation type. Enter your stay dates and check the conditions of your selected option.), List(Children & Beds, Child policies Children over 5 are welcome. To see correct prices and occupancy info, add the number and ages of children in your group to your search. Crib and extra bed policies Cribs and extra beds aren't available at this property.), List(No age restriction, There's no age requirement for check-in), List(Pets, Pets are allowed on request. Charges may apply.), List(Payments by Booking.com, Booking.com takes your payment for this stay on behalf of the property, but make sure you have cash for any extras once you get there.), List(Smoking, Smoking is not allowed.), List(Parties, Parties/events are not allowed), List(Quiet hours, Guests need be quiet between 9:00 PM and 6:00 AM.)) List(English, Polish) List(List(1 full bed | 1 twin bed | 2 sofa beds, 5, Two-Bedroom Apartment)) 9.8 null 48 List(List(gb, Lesley, Beautiful, spacious and very well equipped living space - the hostess had thought of everything you might need
Friendly and responsive hostess 
Quiet location), List(ee, Roman, Very comfortable home stay, will definately come again), List(fr, Xavier, Everything was perfect, well decorated, clean, with a huge bathroom, a nice kitchen and 2 comfy bedrooms.), List(pl, Stefan, Nice appartment in new house in suburbs of Gliwice. Private free parking, nice garden. Host was very welcoming and nice. -), List(pl, Goran, 1. Welcome dinner prepared;
2. Clean and tidy;
3. Garage offered; 
4. Quite and communicative place (closet to A1 highway);
5. Contacting you before arrival; 
6. Any kind of vanity sets available; 
7. Strongly recommended. Everything was excellent.), List(cz, Denisa, Klidná lokalita na okraji obce. Naprosto luxusní ubytování, skvěle vybavené. Pohodlné postele v oddělených ložnicích. Je super mít soukromí.
Prostorná krásná koupelna s vanou i sprchovým koutem. Cítili jsme se snad líp než doma. 😊😉 Paní majitelka byla velice 

count_attractions,count_cafes,count_coworking,count_museums,count_nightlife,count_parks_playgrounds,count_pharmacies,count_restaurants,count_supermarkets,grid_id,grid_lat,grid_lon,min_dist_attractions,min_dist_cafes,min_dist_coworking,min_dist_museums,min_dist_nightlife,min_dist_parks_playgrounds,min_dist_pharmacies,min_dist_restaurants,min_dist_supermarkets
5,55,0,14,15,58,14,110,38,1031_4665,41.909948456731364,12.452050966503547,407.0389752309149,41.74892695746815,null,411.8914551385195,23.3515835881438,143.04158551495254,119.41777498035309,11.362537115685626,79.06444594041352
4,29,0,0,1,4,1,64,4,5568_2804,25.19202328360665,55.280871035420496,389.86028782307466,169.85382741875907,null,null,297.72696380310526,392.51041626647367,127.2910866153603,133.52186396782363,141.05261177859663
2,20,0,0,10,5,13,43,33,1708_4438,39.87196281696668,19.99811947860087,391.87738653868365,177.3337686150511,null,null,176.5283513279118,421.3363820704234,269.2073670749508,79.5968919570971,207.15845287604589
0,0,0,0,0,0,0,0,0,-8295_3381,30.37694154221444,-86.3670273163289,null,null,null,null,null,null,null,null,null
0,25,0,1,3,29,4,11,20,1217_4911,44.1198711150295,15.235810953063543,null,164.09622910411272,null,794.3900253445852,308.47120290637906,299.0474921693966,269.10129820425766,61.075687884002924,133.1685934156732


## 3) Recompute `grid_id` on Booking Listings (Exact Stage 3 Logic)

In [0]:
booking_with_grid = (
    booking
    .withColumn("lat_rad", F.radians("lat"))
    .withColumn("grid_y", F.floor(F.col("lat") * (METERS_PER_DEG_LAT / GRID_M)).cast("long"))
    .withColumn(
        "grid_x",
        F.floor(F.col("lon") * (METERS_PER_DEG_LAT * F.cos(F.col("lat_rad")) / GRID_M)).cast("long")
    )
    .withColumn("grid_id", F.concat_ws("_", "grid_x", "grid_y"))
    .drop("lat_rad")
)

display(booking_with_grid.select("hotel_id", "lat", "lon", "grid_x", "grid_y", "grid_id").limit(10))
print("Unique grids in booking:", booking_with_grid.select("grid_id").distinct().count())


hotel_id,lat,lon,grid_x,grid_y,grid_id
8908679,50.3330617,18.7037022,1329,5603,1329_5603
2246827,23.973931951127646,121.61510512232768,12370,2668,12370_2668
12275653,42.460103063041,18.509718935529,1520,4726,1520_4726
7877735,53.011607650726,18.617122400664,1246,5901,1246_5901
1985847,-18.2022428262032,144.572267532349,15288,-2027,15288_-2027
6921514,47.9016024,1.9014035,141,5332,141_5332
4767573,33.6760868104757,135.3375215895768,12537,3748,12537_3748
291872,46.08231778700002,11.102725267410278,857,5129,857_5129
1383532,4.641235779558954,-75.57255568846101,-8386,516,-8386_516
11140846,35.4390352,133.3418563,12093,3945,12093_3945


Unique grids in booking: 673318


## 5) Join Listing Rows with Grid Enrichment

In [0]:
joined = (
    booking_with_grid.alias("b")
    .join(enrich.alias("e"), on="grid_id", how="left")
    .withColumn("has_enrichment", F.when(F.col("e.grid_lat").isNotNull(), F.lit(1)).otherwise(F.lit(0)))
)

coverage = joined.agg(F.avg("has_enrichment").alias("coverage")).collect()[0]["coverage"]
print("Coverage fraction:", float(coverage))

display(joined.select("hotel_id", "grid_id", "has_enrichment").limit(10))


Coverage fraction: 0.15657881373381602


hotel_id,grid_id,has_enrichment
8908679,1329_5603,0
2246827,12370_2668,0
12275653,1520_4726,1
7877735,1246_5901,1
1985847,15288_-2027,0
6921514,141_5332,0
4767573,12537_3748,0
291872,857_5129,0
1383532,-8386_516,0
11140846,12093_3945,0


## 6) Feature Engineering (Numeric, Interpretable, ML-Friendly)

In [0]:
COUNT_COLS = [c for c in joined.columns if c.startswith("count_")]
DIST_COLS  = [c for c in joined.columns if c.startswith("min_dist_")]

print("COUNT_COLS:", COUNT_COLS)
print("DIST_COLS:", DIST_COLS)

df = joined

# 1) Fill counts null -> 0
for c in COUNT_COLS:
    df = df.withColumn(c, F.coalesce(F.col(c), F.lit(0)).cast("long"))

# 2) Cap distances at RADIUS_M (keep nulls as nulls)
for c in DIST_COLS:
    df = df.withColumn(
        c,
        F.when(F.col(c).isNull(), F.lit(None).cast("double"))
         .otherwise(F.least(F.col(c).cast("double"), F.lit(RADIUS_M)))
    )

# 3) log1p counts
for c in COUNT_COLS:
    df = df.withColumn(f"log1p_{c}", F.log1p(F.col(c).cast("double")))

# Helper for flags
def proximity_flag(dist_col: str, thr: float):
    return (
        F.when(
            (F.col("has_enrichment") == 1) &
            F.col(dist_col).isNotNull() &
            (F.col(dist_col) <= F.lit(thr)),
            F.lit(1)
        ).otherwise(F.lit(0)).cast("int")
    )

# 4) Proximity flags + 5) inverse distance
for d in DIST_COLS:
    base = d.replace("min_dist_", "")  # e.g., cafes, supermarkets, parks_playgrounds

    # Proximity flags
    for thr in PROX_THRESHOLDS_M:
        df = df.withColumn(f"{base}_within_{int(thr)}m", proximity_flag(d, thr))

    # Inverse distance (0 if null or not enriched)
    df = df.withColumn(
        f"inv_{base}_dist",
        F.when((F.col("has_enrichment") == 1) & F.col(d).isNotNull(), 1.0 / (1.0 + F.col(d)))
         .otherwise(F.lit(0.0))
         .cast("double")
    )


display(df.select(
    "hotel_id", "grid_id", "has_enrichment",
    *COUNT_COLS,
    *[c for c in df.columns if c.startswith("log1p_count_")][:5]  # preview a few
).limit(5))


COUNT_COLS: ['count_attractions', 'count_cafes', 'count_coworking', 'count_museums', 'count_nightlife', 'count_parks_playgrounds', 'count_pharmacies', 'count_restaurants', 'count_supermarkets']
DIST_COLS: ['min_dist_attractions', 'min_dist_cafes', 'min_dist_coworking', 'min_dist_museums', 'min_dist_nightlife', 'min_dist_parks_playgrounds', 'min_dist_pharmacies', 'min_dist_restaurants', 'min_dist_supermarkets']


hotel_id,grid_id,has_enrichment,count_attractions,count_cafes,count_coworking,count_museums,count_nightlife,count_parks_playgrounds,count_pharmacies,count_restaurants,count_supermarkets,log1p_count_attractions,log1p_count_cafes,log1p_count_coworking,log1p_count_museums,log1p_count_nightlife
8908679,1329_5603,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2246827,12370_2668,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
12275653,1520_4726,1,0,27,0,0,5,6,3,18,18,0.0,3.332204510175204,0.0,0.0,1.791759469228055
7877735,1246_5901,1,22,31,0,19,13,19,9,100,22,3.1354942159291497,3.4657359027997265,0.0,2.995732273553991,2.639057329615259
1985847,15288_-2027,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0


## 7) Save Outputs

In [0]:
OUT_FULL = f"{OUT_BASE}/booking_listing_features_full"
OUT_ENRICHED = f"{OUT_BASE}/booking_listing_features_enriched"

# Save full (overwrite)
df.write.mode("overwrite").parquet(OUT_FULL)
print("Saved FULL dataset to:", OUT_FULL)

# Save enriched-only subset
df_enriched = df.filter(F.col("has_enrichment") == 1)
df_enriched.write.mode("overwrite").parquet(OUT_ENRICHED)
print("Saved ENRICHED dataset to:", OUT_ENRICHED)

print("Enriched rows:", df_enriched.count())


Saved FULL dataset to: dbfs:/tmp/booking_stage4/booking_listing_features_full
Saved ENRICHED dataset to: dbfs:/tmp/booking_stage4/booking_listing_features_enriched
Enriched rows: 507220


## 8) Sanity Checks (Quick)

In [0]:
full = spark.read.parquet(OUT_FULL)
enr  = spark.read.parquet(OUT_ENRICHED)

display(full.select("has_enrichment").groupBy("has_enrichment").count())
display(enr.select("hotel_id", "grid_id", "count_cafes", "min_dist_cafes").limit(5))


has_enrichment,count
1,507220
0,2732171


hotel_id,grid_id,count_cafes,min_dist_cafes
11505077,531_6112,12,15.89462229269765
7582063,-149_5414,8,77.98197577344027
51504,16_5501,7,63.93609068902704
4763541,566_4848,25,43.35349802596985
427228,-395_4087,92,91.82577124742333


In [0]:
from pyspark.sql import functions as F


ENRICH_COUNT_COLS = [c for c in full.columns if c.startswith("count_")]
ENRICH_LOG_COLS   = [c for c in full.columns if c.startswith("log1p_count_")]
ENRICH_WITHIN_COLS = [c for c in full.columns if c.endswith("_within_200m")
                                           or c.endswith("_within_500m")]
ENRICH_INV_DIST_COLS = [c for c in full.columns if c.startswith("inv_") and c.endswith("_dist")]

# Optional spatial metadata
OPTIONAL_GRID_COLS = ["grid_id"]  # add grid_lat/grid_lon if you want


SCRAPED_FEATURE_COLS = (
    ["hotel_id", "has_enrichment"] +
    OPTIONAL_GRID_COLS +
    ENRICH_COUNT_COLS +
    ENRICH_LOG_COLS +
    ENRICH_WITHIN_COLS +
    ENRICH_INV_DIST_COLS
)

print("Number of scraped ML features:", len(SCRAPED_FEATURE_COLS) - 1)
print("Preview feature names:")
print(SCRAPED_FEATURE_COLS[:15], "...")


scraped_features_ml = full.select(*SCRAPED_FEATURE_COLS)

print("Rows:", scraped_features_ml.count())
display(scraped_features_ml.limit(10))


Number of scraped ML features: 47
Preview feature names:
['hotel_id', 'has_enrichment', 'grid_id', 'count_attractions', 'count_cafes', 'count_coworking', 'count_museums', 'count_nightlife', 'count_parks_playgrounds', 'count_pharmacies', 'count_restaurants', 'count_supermarkets', 'log1p_count_attractions', 'log1p_count_cafes', 'log1p_count_coworking'] ...
Rows: 3239391


hotel_id,has_enrichment,grid_id,count_attractions,count_cafes,count_coworking,count_museums,count_nightlife,count_parks_playgrounds,count_pharmacies,count_restaurants,count_supermarkets,log1p_count_attractions,log1p_count_cafes,log1p_count_coworking,log1p_count_museums,log1p_count_nightlife,log1p_count_parks_playgrounds,log1p_count_pharmacies,log1p_count_restaurants,log1p_count_supermarkets,attractions_within_200m,attractions_within_500m,cafes_within_200m,cafes_within_500m,coworking_within_200m,coworking_within_500m,museums_within_200m,museums_within_500m,nightlife_within_200m,nightlife_within_500m,parks_playgrounds_within_200m,parks_playgrounds_within_500m,pharmacies_within_200m,pharmacies_within_500m,restaurants_within_200m,restaurants_within_500m,supermarkets_within_200m,supermarkets_within_500m,inv_attractions_dist,inv_cafes_dist,inv_coworking_dist,inv_museums_dist,inv_nightlife_dist,inv_parks_playgrounds_dist,inv_pharmacies_dist,inv_restaurants_dist,inv_supermarkets_dist
8908679,0,1329_5603,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2246827,0,12370_2668,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12275653,1,1520_4726,0,27,0,0,5,6,3,18,18,0.0,3.332204510175204,0.0,0.0,1.791759469228055,1.9459101490553132,1.3862943611198906,2.9444389791664403,2.9444389791664403,0,0,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0.0,0.01530368413563394,0.0,0.0,0.00582731449817561,0.006326631732567541,0.013401276375506721,0.008428397747031538,0.012983082492830348
7877735,1,1246_5901,22,31,0,19,13,19,9,100,22,3.1354942159291497,3.4657359027997265,0.0,2.995732273553991,2.639057329615259,2.995732273553991,2.302585092994046,4.61512051684126,3.1354942159291497,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0.008889250563387234,0.01812233736160431,0.0,0.00911338100650738,0.025711411814431716,0.006044187366090331,0.04401203670887594,0.016028951121486317,0.019141423154604528
1985847,0,15288_-2027,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6921514,0,141_5332,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4767573,0,12537_3748,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
291872,0,857_5129,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1383532,0,-8386_516,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11140846,0,12093_3945,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
OUT_SCRAPED_FEATURES = "dbfs:/tmp/booking_stage4/scraped_enrichment_features_ml"

scraped_features_ml.write.mode("overwrite").parquet(OUT_SCRAPED_FEATURES)

print("Saved scraped enrichment ML feature table to:")
print(OUT_SCRAPED_FEATURES)


Saved scraped enrichment ML feature table to:
dbfs:/tmp/booking_stage4/scraped_enrichment_features_ml
